# This notebook contains the code for section 3.3, 3.5, and 3.6
In particular, first we have the Hagen-Poissiulle example, then the Kepler example, then the Einstein example.

Note: if you want to run all three examples, be sure to restart the notebook after running each example, since some examples repurpose variable names, and otherwise the compiler might get confused.

You will also need to have installed Julia, Gurobi and Mosek to reproduce the code run here.

We ran everything using Julia version 1.7.3, Mosek version 9.5 and Gurobi version 9.5.1

Note that the Kepler and Einstein examples do take a little while to run.

In [1]:
using DynamicPolynomials
using JuMP
using MosekTools
using LinearAlgebra
using CSV
using DataFrames
import JSON
using Dates
using Gurobi

# Functions for generating monomials

We have two versions of these functions: one which only controls the overall degree, and one which also controls each variable's degree, in order to have finer grained control

In [2]:
function all_monomials_up_to_max_deg(x, deg)
    if size(x,1) == 0
        [1]
    else
    [ x[1]^k * m for k=0:deg 
            for m=all_monomials_up_to_max_deg(x[2:end], deg)
    ]
    end
end

function mons_of_max_degree_and_unit(x, deg, u)
    [m
        for m=all_monomials_up_to_max_deg(x, deg)
        #if all(unit(m) .== u)
    ]
end
                
function degree_poly(p)
    maximum(degree.(monomials(p)))
end

function all_monomials_up_to_max_deg_overall(x, deg, deg_overall)
    if size(x,1) == 0
        [1]
    else
    [ x[1]^k * m for k=0:min(deg, deg_overall) 
                for m=all_monomials_up_to_max_deg_overall(x[2:end], deg, deg_overall-k)
    ]
    end
end

function mons_of_max_degree_and_unit_overall(x, deg, deg_overall, u)
    [m
        for m=all_monomials_up_to_max_deg_overall(x, deg, deg_overall)
        #if all(unit(m) .== u)
    ]
end

mons_of_max_degree_and_unit_overall (generic function with 1 method)

# Hagen-Poiseuille (First example in paper)

Let us first attempt to derive the Hagen-Poissuille equation

In [ ]:
@polyvar r c_0 c_2 L μ Δp R u dp  # c_1 

x =[r c_0 c_2 L μ Δp R u dp] # c_1 

1×9 Matrix{PolyVar{true}}:
 r  c_0  c_2  L  μ  Δp  R  u  dp

In [ ]:
ur = c_0+c_2*r^2 

r²c_2 + c_0

In [ ]:
ur_fin=differentiate(r*differentiate(ur, r),r)

4rc_2

In [ ]:
bc=subs(ur, r=>R)

c_2R² + c_0

In [ ]:
axioms= 
[
    μ*ur_fin-r*dp, 
    dp*L+Δp, 
    bc,
    u-c_0-c_2*r^2
]

4-element Vector{Polynomial{true, Int64}}:
 4rc_2μ - rdp
 Ldp + Δp
 c_2R² + c_0
 -r²c_2 - c_0 + u

In [ ]:
deg = 3
deg_overall=6
candidate_mons = [
    mons_of_max_degree_and_unit_overall(x, deg, deg_overall, [])
    for ai=axioms
]
@show size.(candidate_mons)

model = Model(Mosek.Optimizer)
mons_q = mons_of_max_degree_and_unit_overall([r R u L μ Δp] , deg, deg_overall, []) # 
coeff_q =   @variable(model, [1:size(mons_q,1)], base_name="q")

q = sum(ci .* mi for (ci, mi)=zip(coeff_q, mons_q)) # Zip pairs things without needing a ref index, e.g., zip([1, 2, 3], [4,5,6])=((1,4), (2,5), (3,6))
coeff_αs = [
    @variable(model, [1:size(X,1)], base_name="α$i")
    for (i,X)=enumerate(candidate_mons)
    ]
@show size.(coeff_αs)
αs = [sum(ci .* mi) for (ci, mi)=zip(coeff_αs, candidate_mons)]

residual = q - sum(αᵢ * aᵢ for (αᵢ, aᵢ)=zip(αs,axioms));
eqs = coefficients(residual)

# Ensure that the sum of the coefficients on the terms involving u isn't zero, in order that u is part of expression
@constraint model sum(coeff_q[degree.(mons_q, u).>0]) == 4.0

@constraint model eqs .== 0
@objective model Max 0

optimize!(model)
@show termination_status(model)


size.(candidate_mons) = [(4510,), (4510,), (4510,), (4510,)]
size.(coeff_αs) = [(4510,), (4510,), (4510,), (4510,)]
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : LO (linear optimization problem)
  Constraints            : 22000           
  Cones                  : 0               
  Scalar variables       : 18796           
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 3837
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.01            
Lin. dep.  - number                 : 82

OPTIMAL::TerminationStatusCode = 1

In [ ]:
value_poly = p -> sum(value.(coefficients(p)).* monomials(p))

#31 (generic function with 1 method)

In [ ]:
value_q = value_poly(q)
value_q

r³Δp + 4.0rLμu - rΔpR²

In [ ]:
@show value_αs = value_poly.(αs)

value_αs = value_poly.(αs) = Polynomial{true, Float64}[r²L - LR², r³ - rR², 4.0rLμ, 4.0rLμ]


4-element Vector{Polynomial{true, Float64}}:
 r²L - LR²
 r³ - rR²
 4.0rLμ
 4.0rLμ

Re-arranging: 

$ u=\frac{-Δp (r^2-R^2)}{4μL}$ as required.

# Kepler example: Distinguishing between axioms via experimental data

Let us revisit Kepler's law, but with irrelevant axioms and experimental data, to aim to distinguish between them

In [3]:
function all_monomials_up_to_max_deg(x, deg)
    if size(x,1) == 0
        [1]
    else
    [ x[1]^k * m for k=0:deg 
            for m=all_monomials_up_to_max_deg(x[2:end], deg)
    ]
    end
end

function mons_of_max_degree_and_unit(x, deg, u)
    [m
        for m=all_monomials_up_to_max_deg(x, deg)
        #if all(unit(m) .== u)
    ]
end

function all_monomials_up_to_max_deg_overall(x, deg, deg_overall)
    if size(x,1) == 0
        [1]
    else
    [ x[1]^k * m for k=0:min(deg, deg_overall) 
                for m=all_monomials_up_to_max_deg_overall(x[2:end], deg, deg_overall-k)
    ]
    end
end

function mons_of_max_degree_and_unit_overall(x, deg, deg_overall, u)
    [m
        for m=all_monomials_up_to_max_deg_overall(x, deg, deg_overall)
        #if all(unit(m) .== u)
    ]
end
                
function degree_poly(p)
    maximum(degree.(monomials(p)))
end

degree_poly (generic function with 1 method)

In [4]:
@polyvar m₁ m₂ d₁ d₂ p w Fg Fc G
x =[m₁, m₂, d₁, d₂, p, w, Fg, Fc, G]

9-element Vector{Variable{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}}}:
 m₁
 m₂
 d₁
 d₂
 p
 w
 Fg
 Fc
 G

In [5]:
# Including some irrelevant axioms here
axioms2 = [d₁*m₁ - d₂*m₂,
          (d₁+d₂)^2*Fg-G*m₁*m₂,
          p^2*m₁-0.1319*(d₁+d₂)^3, # One of the (incorrect) axioms from the AI-Descartes paper
          Fc-m₂*d₂*w^2,
          Fg-Fc,
          w*p-1
         ]

6-element Vector{Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}, Float64}}:
 -m₂d₂ + m₁d₁
 d₂²Fg + 2.0d₁d₂Fg + d₁²Fg - m₁m₂G
 -0.1319d₂³ - 0.39569999999999994d₁d₂² - 0.39569999999999994d₁²d₂ - 0.1319d₁³ + m₁p²
 Fc - m₂d₂w²
 -Fc + Fg
 -1.0 + pw

In [6]:
data_bin_star=[
    # Period (years) # m1 #m2 #d1 #d2 # G
    1089.0 0.54 0.50 0.0 0.0 0.0
    143.1  1.33 1.41 0.0 0.0 0.0
    930.0  0.88 0.82 0.0 0.0 0.0
    675.5  3.06 1.97 0.0 0.0 0.0
]
d=[107.270 38.235 113.769 131.352]
for i=1:4
   data_bin_star[i,4]=d[i]*data_bin_star[i,3]/(data_bin_star[i,2]+data_bin_star[i,3]) 
    data_bin_star[i,5]=d[i]*data_bin_star[i,2]/(data_bin_star[i,2]+data_bin_star[i,3]) 
end
data_bin_star=data_bin_star[:,[2,3,4,5,1,6]] # Move period to end, to be consistent with model

data_bin_star[:,5].=data_bin_star[:,5]/(2*pi) # Scale out 2*pi factor

theG=4*pi^2 # In solar system units
                

data_bin_star[:,6].=theG;


In [7]:
data_bin_star

4×6 Matrix{Float64}:
 0.54  0.5   51.5721  55.6979  173.32    39.4784
 1.33  1.41  19.6757  18.5593   22.7751  39.4784
 0.88  0.82  54.8768  58.8922  148.014   39.4784
 3.06  1.97  51.444   79.908   107.509   39.4784

In [12]:
### deg = 2
deg_overall=6
deg_overall_q=5

deg_elementwise=[2, 2, 2, 2, 2, 2, 2, 2, 2] 
deg_elementwise_q=[2, 2, 2, 2, 2, 2] 

deg = 2
k=5 # cardinality constraint RHS
M=10

candidate_mons = [
    mons_of_max_degree_and_unit_overall(x, deg, deg_overall, deg_elementwise, [])
    for ai=axioms2
]
@show size.(candidate_mons)

model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "Presolve", 0)
set_optimizer_attribute(model, "OptimalityTol", 1e-9)
set_optimizer_attribute(model, "FeasibilityTol", 1e-9)
set_optimizer_attribute(model, "IntFeasTol", 1e-9)
set_optimizer_attribute(model, "MIPGap", 1e-9)
set_optimizer_attribute(model, "Quad", 1)


mons_q = mons_of_max_degree_and_unit_overall([p, m₁, m₂, d₁, d₂, G] , deg, deg_overall_q, deg_elementwise_q, [])
coeff_q =   @variable(model, [1:size(mons_q,1)], base_name="q")
abs_coeff_q =   @variable(model, [1:size(mons_q,1)], base_name="q")

q = sum(ci .* mi for (ci, mi)=zip(coeff_q, mons_q)) # Zip pairs things without needing a ref index, e.g., zip([1, 2, 3], [4,5,6])=((1,4), (2,5), (3,6))
coeff_αs = [
    @variable(model, [1:size(X,1)], base_name="α$i")
    for (i,X)=enumerate(candidate_mons)
    ]
@show size.(coeff_αs)
αs = [sum(ci .* mi) for (ci, mi)=zip(coeff_αs, candidate_mons)]

residual = q - sum(αᵢ * aᵢ for (αᵢ, aᵢ)=zip(αs,axioms2));
eqs = coefficients(residual)


@variable(model, z[i=1:size(axioms2,1)], Bin)

# @constraint(model, imposeLogical1[i=1:size(axioms2,1)], coefficients(αs[i]).<= M*z[i]) 
# @constraint(model, imposeLogical2[i=1:size(axioms2,1)], -coefficients(αs[i]).<= M*z[i]) 
@constraint(model, imposeLogical[i=1:size(axioms2,1)], !(z[i]) .=> {coefficients(αs[i]).==0.0}) #logical 


@constraint(model, sum(z)<=k)

# Ensure that the sum of the coefficients on the terms involving p isn't zero, in order that p is part of expression
@constraint model sum(coeff_q[degree.(mons_q, p).>0]) == 1.0

# @constraint model eqs .== 0
@constraint model abs_coeff_q.>=coeff_q
@constraint model abs_coeff_q.>=-coeff_q
@variable(model, t[i=1:4]>=0.0)
@constraint(model, imposeabs1[i=1:4], t[i]>=q(data_bin_star[i,:]))
@constraint(model, imposeabs2[i=1:4], t[i]>=-q(data_bin_star[i,:]))
@objective model Min 100*sum(t)+sum(abs_coeff_q)+1000*sum(abs.(eqs)) # Reduce number of non-zero coefficients via Lasso trick

optimize!(model)
@show termination_status(model)


size.(candidate_mons) = [(3061,), (3061,), (3061,), (3061,), (3061,), (3061,)]
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-13
Set parameter Presolve to value 0
Set parameter OptimalityTol to value 1e-09
Set parameter FeasibilityTol to value 1e-09
Set parameter IntFeasTol to value 1e-09
Set parameter MIPGap to value 1e-09
Set parameter Quad to value 1
size.(coeff_αs) = [(3061,), (3061,), (3061,), (3061,), (3061,), (3061,)]
size(eqs) = (25063,)
Set parameter FeasibilityTol to value 1e-09
Set parameter MIPGap to value 1e-09
Set parameter Quad to value 1
Set parameter OptimalityTol to value 1e-09
Set parameter IntFeasTol to value 1e-09
Set parameter Presolve to value 0
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i9-8950HK CPU @ 2.90GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 50724 rows, 44027 columns and 158477 nonzeros
Model fingerprint: 

OPTIMAL::TerminationStatusCode = 1

In [72]:
@show value.(t)

value.(t) = [255.01255520191626, 145.3171148051224, 944.8754072092124, 34669.34502677247]


4-element Vector{Float64}:
   255.01255520191626
   145.3171148051224
   944.8754072092124
 34669.34502677247

In [73]:
@show value.(z)

value.(z) = [1.0, 1.0, 0.0, 1.0, 1.0, 1.0]


6-element Vector{Float64}:
 1.0
 1.0
 0.0
 1.0
 1.0
 1.0

In [77]:
value_poly2 = p -> sum(value.(coefficients(p)).* monomials(p))
@show value_poly2(q)


@show value_αs=value_poly2.(αs)




value_poly2(q) = m₁m₂p²G - m₁d₁d₂² - m₂d₁²d₂ - 2.0m₂d₁d₂²
value_αs = value_poly2.(αs) = Polynomial{true, Float64}[-d₂²p²w², -p², 0.0, d₁²p² + 2.0d₁d₂p² + d₂²p², d₁²p² + 2.0d₁d₂p² + d₂²p², m₁d₁d₂²pw + m₂d₁²d₂pw + 2.0m₂d₁d₂²pw + m₁d₁d₂² + m₂d₁²d₂ + 2.0m₂d₁d₂²]


6-element Vector{Polynomial{true, Float64}}:
 -d₂²p²w²
 -p²
 0.0
 d₁²p² + 2.0d₁d₂p² + d₂²p²
 d₁²p² + 2.0d₁d₂p² + d₂²p²
 m₁d₁d₂²pw + m₂d₁²d₂pw + 2.0m₂d₁d₂²pw + m₁d₁d₂² + m₂d₁²d₂ + 2.0m₂d₁d₂²

# Einstein: distinguishing between axioms

In [3]:
@polyvar c d dt dt₀ f₀ f L v 
x =[c d dt dt₀ f₀ f L v]

1×8 Matrix{Variable{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}}}:
 c  d  dt  dt₀  f₀  f  L  v

In [4]:
axioms = [
         c*dt₀-2*d,
         c*dt-2*L, #Real axiom
         dt^2*(v^2+c^2)-4*L^2, # Fake "Newtonian" axiom
         4*L^2-4*d^2-v^2*dt^2,
         f₀*dt₀-1,
         f*dt-1
        ]

6-element Vector{Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}, Int64}}:
 -2d + cdt₀
 -2L + cdt
 -4L² + dt²v² + c²dt²
 4L² - 4d² - dt²v²
 -1 + dt₀f₀
 -1 + dtf

In [5]:
data_einstein=Array{BigFloat,2}
data_einstein=[
    #v # f0    # f # c
    0.55 -0.018 0.0 0.0
    4.10 -0.21 0.0 0.0
    8.60 -0.43 0.0 0.0
    14.84 -1.54 0.0 0.0
    22.18 -2.92 0.0 0.0
    29.65 -4.82 0.0 0.0
    36.22 -7.36 0.0 0.0
]
data_einstein[:,2].*=10^-15



# # Second column is originally (f-f_0)/f_0 and has units on the order of 10^-15, need to convert to f0, f
# # Define f, f_0 to be the same order of magnitude
thef0=1.0000

speedLight=3*10^8

data_einstein[:,4].=speedLight
for i=1:7
    data_einstein[i,3]=thef0+thef0*data_einstein[i,2]
    @show thef0*(1.0.+data_einstein[i,2])
    data_einstein[i,2]=thef0
    # Rescale units to be in km, not in m (3*10^8 is not well conditioned, numerically speaking)
    data_einstein[i,1]=data_einstein[i,1]/100.0
    data_einstein[i,4]=data_einstein[i,4]/100.0
end
data_einstein=data_einstein[:, [4, 2, 3, 1]] # Change order of columns to be alphabetical and therefore consistent with polynomials package
    
print(data_einstein)


ground_truth=f₀^2*v^2 + c^2*f^2 - c^2*f₀^2
for i=1:7
    @show ground_truth(data_einstein[i,:])
end

# Data prepared in original variables, in order that we need not introduce any new variables

thef0 * (1.0 .+ data_einstein[i, 2]) = 1.0
thef0 * (1.0 .+ data_einstein[i, 2]) = 0.9999999999999998
thef0 * (1.0 .+ data_einstein[i, 2]) = 0.9999999999999996
thef0 * (1.0 .+ data_einstein[i, 2]) = 0.9999999999999984
thef0 * (1.0 .+ data_einstein[i, 2]) = 0.9999999999999971
thef0 * (1.0 .+ data_einstein[i, 2]) = 0.9999999999999952
thef0 * (1.0 .+ data_einstein[i, 2]) = 0.9999999999999927
[3.0e6 1.0 1.0 0.0055000000000000005; 3.0e6 1.0 0.9999999999999998 0.040999999999999995; 3.0e6 1.0 0.9999999999999996 0.086; 3.0e6 1.0 0.9999999999999984 0.1484; 3.0e6 1.0 0.9999999999999971 0.2218; 3.0e6 1.0 0.9999999999999952 0.2965; 3.0e6 1.0 0.9999999999999927 0.36219999999999997]

In [6]:
function all_monomials_up_to_max_deg_overall(x, deg, deg_overall, deg_elem)
    if size(x,1) == 0
        [1]
    else
    [ x[1]^k * m for k=0:min(deg, deg_overall, deg_elem[1]) 
                for m=all_monomials_up_to_max_deg_overall(x[2:end], deg, deg_overall-k, deg_elem[2:end])
    ]
    end
end

function mons_of_max_degree_and_unit_overall(x, deg, deg_overall, deg_elem, u)
    [m
        for m=all_monomials_up_to_max_deg_overall(x, deg, deg_overall, deg_elem)
        #if all(unit(m) .== u)
    ]
end
                
function degree_poly(p)
    maximum(degree.(monomials(p)))
end

degree_poly (generic function with 1 method)

In [9]:
### deg = 2
deg_overall=10
deg_overall_q=4

deg_elementwise=[2, 2, 2, 2, 2, 2, 2, 2] 
deg_elementwise_q=[2, 2, 2, 2] 

deg = 2
k=5 # cardinality constraint RHS
# M=10 # 

candidate_mons = [
    mons_of_max_degree_and_unit_overall(x, deg, deg_overall, deg_elementwise, [])
    for ai=axioms
]
@show size.(candidate_mons)

model = Model(Gurobi.Optimizer)

set_optimizer_attribute(model, "Presolve", 0)
set_optimizer_attribute(model, "OptimalityTol", 1e-3)
set_optimizer_attribute(model, "FeasibilityTol", 1e-3)
set_optimizer_attribute(model, "IntFeasTol", 1e-3)
set_optimizer_attribute(model, "MIPGap", 1e-3)
set_optimizer_attribute(model, "NodeMethod", 0)
set_optimizer_attribute(model, "Method", 0)
# Note: somewhat loose tolerance, as a very numerically unstable problem
set_optimizer_attribute(model, "NumericFocus", 3)


mons_q = mons_of_max_degree_and_unit_overall([c f₀ f v] , deg, deg_overall_q, deg_elementwise_q, [])
coeff_q =   @variable(model, [1:size(mons_q,1)], base_name="q")
abs_coeff_q =   @variable(model, [1:size(mons_q,1)], base_name="q")

q = sum(ci .* mi for (ci, mi)=zip(coeff_q, mons_q)) # Zip pairs things without needing a ref index, e.g., zip([1, 2, 3], [4,5,6])=((1,4), (2,5), (3,6))
coeff_αs = [
    @variable(model, [1:size(X,1)], base_name="α$i")
    for (i,X)=enumerate(candidate_mons)
    ]
@show size.(coeff_αs)
αs = [sum(ci .* mi) for (ci, mi)=zip(coeff_αs, candidate_mons)]

residual = q - sum(αᵢ * aᵢ for (αᵢ, aᵢ)=zip(αs,axioms));
eqs = coefficients(residual)


@variable(model, z[i=1:size(axioms,1)], Bin)

@constraint(model, imposeLogical[i=1:size(axioms,1)], !(z[i]) .=> {coefficients(αs[i]).==0.0}) #logical 



@constraint(model, sum(z)<=k)

# Ensure that the sum of the coefficients on the terms involving v isn't zero, in order that v is part of expression
@constraint model sum(coeff_q[degree.(mons_q, f).>0]) == 1.0

@constraint model eqs .== 0
@constraint model abs_coeff_q.>=coeff_q
@constraint model abs_coeff_q.>=-coeff_q
# # @objective model Max 0
@variable(model, t[i=1:7]>=0.0)
@constraint(model, imposeabs[i=1:7], t[i]>=q(data_einstein[i,:]))
@constraint(model, imposeabs2[i=1:7], t[i]>=-q(data_einstein[i,:]))
@objective model Min 0.001*sum(abs_coeff_q)+ 10^6*sum(t)

optimize!(model)
@show termination_status(model)

size.(candidate_mons) = [(5634,), (5634,), (5634,), (5634,), (5634,), (5634,)]
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-14
Set parameter Presolve to value 0
Set parameter OptimalityTol to value 0.001
Set parameter FeasibilityTol to value 0.001
Set parameter IntFeasTol to value 0.001
Set parameter MIPGap to value 0.001
Set parameter NodeMethod to value 0
Set parameter Method to value 0
Set parameter NumericFocus to value 3
size.(coeff_αs) = 

In [ ]:
@show value.(t)
@show value.(z)
@show value_poly2(q)

value.(t) = [0.0, 0.0011272430419921875, 0.0018053054809570312, 0.0021076202392578125, 0.0, 0.004910469055175781, 0.0]
value.(z) = [1.0, 1.0, -0.0, 1.0, 1.0, 1.0]
value_poly2(q) = -c²f₀² + c²f² - 9.389298021553431e-15c²v² + f₀²v² - 2.4930024825566218e-17c²f₀ + 3.967856195787149e-15c²v


-c²f₀² + c²f² - 9.389298021553431e-15c²v² + f₀²v² - 2.4930024825566218e-17c²f₀ + 3.967856195787149e-15c²v

In [ ]:
@show value_poly2.(αs)

value_poly2.(αs) = Polynomial{true, Float64}[2.0df₀²f² + cf₀f², -2.0f₀²f²L - cf₀²f, 0.0, -f₀²f², -2.0cdf₀f² - c²f², -dtf₀²fv² + 2.0cf₀²fL + c²f₀² - f₀²v²]


6-element Vector{Polynomial{true, Float64}}:
 2.0df₀²f² + cf₀f²
 -2.0f₀²f²L - cf₀²f
 0.0
 -f₀²f²
 -2.0cdf₀f² - c²f²
 -dtf₀²fv² + 2.0cf₀²fL + c²f₀² - f₀²v²

In [ ]:
### deg = 2, Kepler version with equality constraint relaxed
deg_overall=6
deg_overall_q=5

deg_elementwise=[2, 2, 2, 2, 2, 2, 2, 2, 2] 
deg_elementwise_q=[2, 2, 2, 2, 2, 2] 

deg = 2
k=5 # cardinality constraint RHS
M=10

candidate_mons = [
    mons_of_max_degree_and_unit_overall(x, deg, deg_overall, deg_elementwise, [])
    for ai=axioms2
]
@show size.(candidate_mons)

model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "Presolve", 0)
set_optimizer_attribute(model, "OptimalityTol", 1e-9)
set_optimizer_attribute(model, "FeasibilityTol", 1e-9)
set_optimizer_attribute(model, "IntFeasTol", 1e-9)
set_optimizer_attribute(model, "MIPGap", 1e-9)
set_optimizer_attribute(model, "Quad", 1)


mons_q = mons_of_max_degree_and_unit_overall([p, m₁, m₂, d₁, d₂, G] , deg, deg_overall_q, deg_elementwise_q, [])
coeff_q =   @variable(model, [1:size(mons_q,1)], base_name="q")
abs_coeff_q =   @variable(model, [1:size(mons_q,1)], base_name="q")

q = sum(ci .* mi for (ci, mi)=zip(coeff_q, mons_q)) # Zip pairs things without needing a ref index, e.g., zip([1, 2, 3], [4,5,6])=((1,4), (2,5), (3,6))
coeff_αs = [
    @variable(model, [1:size(X,1)], base_name="α$i")
    for (i,X)=enumerate(candidate_mons)
    ]
@show size.(coeff_αs)
αs = [sum(ci .* mi) for (ci, mi)=zip(coeff_αs, candidate_mons)]

residual = q - sum(αᵢ * aᵢ for (αᵢ, aᵢ)=zip(αs,axioms2));
eqs = coefficients(residual)
@show size(eqs)
@variable(model, abseqs[i=1:size(eqs,1)])
@constraint model abseqs.>=eqs
@constraint model abseqs.>=-eqs


@variable(model, z[i=1:size(axioms2,1)], Bin)

# @constraint(model, imposeLogical1[i=1:size(axioms2,1)], coefficients(αs[i]).<= M*z[i]) 
# @constraint(model, imposeLogical2[i=1:size(axioms2,1)], -coefficients(αs[i]).<= M*z[i]) 
@constraint(model, imposeLogical[i=1:size(axioms2,1)], !(z[i]) .=> {coefficients(αs[i]).==0.0}) #logical 


@constraint(model, sum(z)<=k)

# Ensure that the sum of the coefficients on the terms involving p isn't zero, in order that p is part of expression
@constraint model sum(coeff_q[degree.(mons_q, p).>0]) == 1.0

# @constraint model eqs .== 0
@constraint model abs_coeff_q.>=coeff_q
@constraint model abs_coeff_q.>=-coeff_q
@variable(model, t[i=1:4]>=0.0)
@constraint(model, imposeabs1[i=1:4], t[i]>=q(data_bin_star[i,:]))
@constraint(model, imposeabs2[i=1:4], t[i]>=-q(data_bin_star[i,:]))
@objective model Min 100*sum(t)+sum(abs_coeff_q)+1000*sum(abseqs) # Reduce number of non-zero coefficients via Lasso trick

optimize!(model)
@show termination_status(model)
